## Multi-label Classification Experiments

#### Predicting 'is a' relationship for top 10 and 20 labels

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, make_scorer, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier

In [2]:
go_file = "../dataset/go-basic.obo"

 Parse the data from the dataset

In [3]:
def parse_obo_file(file_path):
    
    data = []
    current_term = {}
    in_term_block = False
    
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line == '[Term]':  #starting a new term block
                if current_term:
                    data.append(current_term)
                current_term = {}
                in_term_block = True
            elif line == '':
                in_term_block = False  #end of a term block
            elif in_term_block:
                if ': ' in line:
                    key, value = line.split(': ', 1)
                    if key in current_term:  #handling multiple lines of the same key
                        if isinstance(current_term[key], list):
                            current_term[key].append(value)
                        else:
                            current_term[key] = [current_term[key], value]
                    else:
                        current_term[key] = value

    
    if current_term: #add the last term if file does not end with a newline
        data.append(current_term)

    return pd.DataFrame(data)

df = parse_obo_file(go_file)


In [4]:
df = df.rename(columns={'def': 'definition'}) 

In [5]:
df.shape

(47856, 14)

In [6]:
df.head(3)

,id,name,namespace,definition,synonym,is_a,alt_id,comment,is_obsolete,replaced_by,consider,xref,subset,relationship
0,GO:0000001,mitochondrion inheritance,biological_process,"""The distribution of mitochondria, including t...","""mitochondrial inheritance"" EXACT []","[GO:0048308 ! organelle inheritance, GO:004831...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GO:0000002,mitochondrial genome maintenance,biological_process,"""The maintenance of the structure and integrit...",NaN,GO:0007005 ! mitochondrion organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GO:0000003,obsolete reproduction,biological_process,"""OBSOLETE. The production of new individuals t...","""reproductive physiological process"" EXACT []",NaN,"[GO:0019952, GO:0050876]",The reason for obsoletion is that this term is...,true,GO:0022414,NaN,NaN,NaN,NaN


In [7]:
df.iloc[0]['is_a']

['GO:0048308 ! organelle inheritance',
 'GO:0048311 ! mitochondrion distribution']

In [8]:
print(df[df['is_a'].isna() == False].shape)
print(df[df['is_a'].isna() == True].shape)

(42200, 14)
(5656, 14)


Total records = 47856

42200 records have is_a relationship present

5656 records do not have is_a relationship present

### Data Preprocessing

#### 1. Excluding the records which do not have is_a

In [9]:
df = df[df['is_a'].notna()]

In [10]:
df[df.is_obsolete.isna() == True]

,id,name,namespace,definition,synonym,is_a,alt_id,comment,is_obsolete,replaced_by,consider,xref,subset,relationship
0,GO:0000001,mitochondrion inheritance,biological_process,"""The distribution of mitochondria, including t...","""mitochondrial inheritance"" EXACT []","[GO:0048308 ! organelle inheritance, GO:004831...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GO:0000002,mitochondrial genome maintenance,biological_process,"""The maintenance of the structure and integrit...",NaN,GO:0007005 ! mitochondrion organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GO:0000006,high-affinity zinc transmembrane transporter a...,molecular_function,"""Enables the transfer of zinc ions (Zn2+) from...","[""high affinity zinc uptake transmembrane tran...",GO:0005385 ! zinc ion transmembrane transporte...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,GO:0000007,low-affinity zinc ion transmembrane transporte...,molecular_function,"""Enables the transfer of a solute or solutes f...",NaN,GO:0005385 ! zinc ion transmembrane transporte...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,GO:0000009,"alpha-1,6-mannosyltransferase activity",molecular_function,"""Catalysis of the transfer of a mannose residu...","""1,6-alpha-mannosyltransferase activity"" EXACT []",GO:0000030 ! mannosyltransferase activity,NaN,NaN,NaN,NaN,NaN,"Reactome:R-HSA-449718 ""Addition of a third man...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47851,GO:2001313,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"""The chemical reactions and pathways involving...","""UDP-4-deoxy-4-formamido-beta-L-arabinopyranos...","[GO:0006040 ! amino sugar metabolic process, G...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47852,GO:2001314,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"""The chemical reactions and pathways resulting...","[""UDP-4-deoxy-4-formamido-beta-L-arabinopyrano...",[GO:0009227 ! nucleotide-sugar catabolic proce...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47853,GO:2001315,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,"""The chemical reactions and pathways resulting...","[""UDP-4-deoxy-4-formamido-beta-L-arabinopyrano...",[GO:0009226 ! nucleotide-sugar biosynthetic pr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47854,GO:2001316,kojic acid metabolic process,biological_process,"""The chemical reactions and pathways involving...","[""5-hydroxy-2-(hydroxymethyl)-4H-pyran-4-one m...",[GO:0034308 ! primary alcohol metabolic proces...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Checking how many unique values is_a has

In [11]:
exploded_df = df.explode('is_a')

In [12]:
exploded_df['is_a'].nunique()

15547

#### 15547 unique 'is a' values

#### 2. Converting the is_a values to only have GO term ids instead of names too :)

In [13]:
def extract_go_terms(s):
    go_terms = []

    if isinstance(s, list):
        for item in s:
            go_terms.extend(re.findall(r'GO:\d{7}', item))
    else:
        go_terms = re.findall(r'GO:\d{7}', s)
    return go_terms if len(go_terms) > 1 else (go_terms[0] if go_terms else None)

In [14]:
df.head(2)

,id,name,namespace,definition,synonym,is_a,alt_id,comment,is_obsolete,replaced_by,consider,xref,subset,relationship
0,GO:0000001,mitochondrion inheritance,biological_process,"""The distribution of mitochondria, including t...","""mitochondrial inheritance"" EXACT []","[GO:0048308 ! organelle inheritance, GO:004831...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GO:0000002,mitochondrial genome maintenance,biological_process,"""The maintenance of the structure and integrit...",NaN,GO:0007005 ! mitochondrion organization,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df = df[['id','definition','is_a']]

In [16]:
df['is_a'] = df['is_a'].apply(extract_go_terms)

In [17]:
df

,id,definition,is_a
0,GO:0000001,"""The distribution of mitochondria, including t...","[GO:0048308, GO:0048311]"
1,GO:0000002,"""The maintenance of the structure and integrit...",GO:0007005
4,GO:0000006,"""Enables the transfer of zinc ions (Zn2+) from...",GO:0005385
5,GO:0000007,"""Enables the transfer of a solute or solutes f...",GO:0005385
7,GO:0000009,"""Catalysis of the transfer of a mannose residu...",GO:0000030
...,...,...,...
47851,GO:2001313,"""The chemical reactions and pathways involving...","[GO:0006040, GO:0006793, GO:0009225]"
47852,GO:2001314,"""The chemical reactions and pathways resulting...","[GO:0009227, GO:0046348, GO:2001313]"
47853,GO:2001315,"""The chemical reactions and pathways resulting...","[GO:0009226, GO:0046349, GO:2001313]"
47854,GO:2001316,"""The chemical reactions and pathways involving...","[GO:0034308, GO:0042180, GO:0120254]"


#### 3. Remove list from definitions

In [18]:
df['definition'] = df['definition'].str.replace(r' \[.*?\]$', '', regex=True)

In [19]:
df['definition'][0]

'"The distribution of mitochondria, including the mitochondrial genome, into daughter cells after mitosis or meiosis, mediated by interactions between mitochondria and the cytoskeleton."'

In [20]:
df['is_a'] = df['is_a'].apply(lambda x: x if isinstance(x, list) else [x])

#### 4. Expanding the entries in df for 'is a' into separate rows

This exploded_df to be used to check unique values and their frequencies

In [21]:
exploded_df = df.explode('is_a')

### Experiments for top 10 labels

### Selecting only the records which have is_a as the 10 most frequent values

In [22]:
exploded_df['is_a'].value_counts().head(10)

is_a
GO:0110165    431
GO:0016616    310
GO:0032991    277
GO:0016709    261
GO:0016758    208
GO:0048856    202
GO:0098797    180
GO:0140513    172
GO:0016747    153
GO:0003006    151
Name: count, dtype: int64

In [23]:
df_top10 = df.copy()
is_a_of_interest_top10 = ["GO:0110165","GO:0016616", "GO:0032991", "GO:0016709", "GO:0016758", "GO:0048856", "GO:0098797", "GO:0140513", "GO:0016747", "GO:0003006"]
df_top10['is_a'] = df_top10['is_a'].apply(lambda labels: [label for label in labels if label in is_a_of_interest_top10])
filtered_df_top10 = df_top10[df_top10['is_a'].apply(lambda x: any(item in is_a_of_interest_top10 for item in (x if isinstance(x, list) else [x])))]

#### Converting the labels to vectors for 10 labels

In [24]:
mlb = MultiLabelBinarizer()
y_top10 = mlb.fit_transform(filtered_df_top10['is_a'])
y_top10_df = pd.DataFrame(y_top10, columns=mlb.classes_) 

In [25]:
mlb.classes_

array(['GO:0003006', 'GO:0016616', 'GO:0016709', 'GO:0016747',
       'GO:0016758', 'GO:0032991', 'GO:0048856', 'GO:0098797',
       'GO:0110165', 'GO:0140513'], dtype=object)

In [26]:
y_top10_df

,GO:0003006,GO:0016616,GO:0016709,GO:0016747,GO:0016758,GO:0032991,GO:0048856,GO:0098797,GO:0110165,GO:0140513
0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
2325,0,0,0,0,0,1,0,0,0,0
2326,0,0,0,0,0,0,0,0,1,0
2327,0,0,0,0,0,1,0,0,0,0
2328,0,0,0,0,0,1,0,0,0,0


In [27]:
y_top10_df.columns

Index(['GO:0003006', 'GO:0016616', 'GO:0016709', 'GO:0016747', 'GO:0016758',
       'GO:0032991', 'GO:0048856', 'GO:0098797', 'GO:0110165', 'GO:0140513'],
      dtype='object')

#### Converting definition attribute to feature vectors for 10 labels

Excluding the words which appear in less than 1% of definitions

In [28]:
vectorizer = CountVectorizer(stop_words='english', min_df = 0.01)
X_tfidf = vectorizer.fit_transform(filtered_df_top10['definition'])
X_df_top10 = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())


In [29]:
X_df_top10

,11,12,acceptor,acetyl,acid,actin,activation,activity,acyl,alpha,...,time,tissue,transcription,transfer,transmembrane,transport,type,udp,wall,yeast
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2325,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2326,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2327,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2328,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Random Forests

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_df_top10, y_top10_df, test_size=0.2, random_state=42)

In [31]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("F1-Score:", str(f1_score(y_test, y_pred_rf, average = 'micro')))
print(classification_report(y_test, y_pred_rf, zero_division=0))

F1-Score: 0.8606741573033708
              precision    recall  f1-score   support

           0       0.79      0.61      0.69        38
           1       1.00      1.00      1.00        76
           2       1.00      0.96      0.98        50
           3       1.00      0.94      0.97        16
           4       0.97      0.91      0.94        35
           5       0.77      0.72      0.74        46
           6       0.96      0.89      0.92        53
           7       1.00      0.44      0.61        32
           8       0.90      0.82      0.86        98
           9       0.65      0.56      0.60        27

   micro avg       0.91      0.81      0.86       471
   macro avg       0.90      0.78      0.83       471
weighted avg       0.91      0.81      0.85       471
 samples avg       0.82      0.82      0.82       471



Random Forest Optimization

In [35]:
rf_model = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': list(range(20,300,50)), 
    'max_depth': list(range(3,60,3)), 
    'min_samples_split': [2, 5, 10, 20, 30], 
    'min_samples_leaf': [1, 2, 4, 10]
}
scoring = {'F1-Score': make_scorer(f1_score, average='micro')}
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring=scoring, refit='F1-Score', n_jobs=-1)

grid_search.fit(X_train, y_train)
print('Best parameters found: \n', grid_search.best_params_)

/Users/kajolpatel/Desktop/Individual_Project/MultiGOClassifier/.venv/lib/python3.11/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best parameters found: 
 {'max_depth': 42, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 170}


In [42]:
grid_search.best_estimator_

RandomForestClassifier(max_depth=42, n_estimators=170, random_state=42)

Testing tuned parameters

In [32]:
rf_model = RandomForestClassifier(max_depth=42, n_estimators=170, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("F1-Score:", str(f1_score(y_test, y_pred_rf, average = 'micro')))
print(classification_report(y_test, y_pred_rf, zero_division=0))

F1-Score: 0.8676307007786429
              precision    recall  f1-score   support

           0       0.83      0.63      0.72        38
           1       1.00      0.99      0.99        76
           2       1.00      0.96      0.98        50
           3       1.00      0.94      0.97        16
           4       0.97      0.91      0.94        35
           5       0.70      0.65      0.67        46
           6       0.96      0.89      0.92        53
           7       1.00      0.50      0.67        32
           8       0.90      0.89      0.89        98
           9       0.70      0.59      0.64        27

   micro avg       0.91      0.83      0.87       471
   macro avg       0.90      0.79      0.84       471
weighted avg       0.91      0.83      0.86       471
 samples avg       0.83      0.83      0.83       471



#### Support Vector Machines

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_df_top10, y_top10_df, test_size=0.2, random_state=42)

In [34]:
svm_model = OneVsRestClassifier(SVC(random_state=42))
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
print(f1_score(y_test, y_pred_svm, average='micro'))
print(classification_report(y_test, y_pred_svm, zero_division=0))

0.8429378531073446
              precision    recall  f1-score   support

           0       0.92      0.58      0.71        38
           1       1.00      0.97      0.99        76
           2       1.00      0.92      0.96        50
           3       0.78      0.88      0.82        16
           4       0.97      0.80      0.88        35
           5       0.64      0.59      0.61        46
           6       0.96      0.91      0.93        53
           7       1.00      0.44      0.61        32
           8       0.86      0.89      0.87        98
           9       0.81      0.48      0.60        27

   micro avg       0.90      0.79      0.84       471
   macro avg       0.89      0.74      0.80       471
weighted avg       0.90      0.79      0.83       471
 samples avg       0.80      0.79      0.79       471



SVM Optimization

In [46]:
svm_model = OneVsRestClassifier(SVC(random_state=42))

param_grid = {
    'estimator__C': [0.1, 1, 10, 0.01, 0.5, 5, 50],
    'estimator__kernel': ['linear', 'rbf', 'poly'],
    'estimator__gamma': ['scale', 'auto',0.001, 0.01, 0.1, 1, 10]
}

scoring = {'F1-Score': make_scorer(f1_score, average='micro')}
grid_search = GridSearchCV(svm_model, param_grid, scoring=scoring, refit='F1-Score', cv=5, n_jobs=-1)

# Perform the search
grid_search.fit(X_train, y_train)

# Best parameter set
print('Best parameters found: \n', grid_search.best_params_)

Best parameters found: 
 {'estimator__C': 50, 'estimator__gamma': 0.01, 'estimator__kernel': 'rbf'}


Testing tuned parameters

In [35]:
svm_model = OneVsRestClassifier(SVC(C= 50, gamma = 0.01, random_state=42))
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
print(f1_score(y_test, y_pred_svm, average='micro'))
print(classification_report(y_test, y_pred_svm, zero_division=0))


0.8512486427795874
              precision    recall  f1-score   support

           0       0.89      0.66      0.76        38
           1       0.99      1.00      0.99        76
           2       1.00      0.94      0.97        50
           3       0.88      0.94      0.91        16
           4       0.92      1.00      0.96        35
           5       0.62      0.65      0.64        46
           6       0.94      0.94      0.94        53
           7       0.89      0.53      0.67        32
           8       0.86      0.83      0.84        98
           9       0.55      0.59      0.57        27

   micro avg       0.87      0.83      0.85       471
   macro avg       0.86      0.81      0.83       471
weighted avg       0.87      0.83      0.85       471
 samples avg       0.82      0.83      0.82       471



### Experiments for top 20 labels

### Selecting only the records which have is_a as the 20 most frequent values

In [36]:
exploded_df['is_a'].value_counts().head(20)

is_a
GO:0110165    431
GO:0016616    310
GO:0032991    277
GO:0016709    261
GO:0016758    208
GO:0048856    202
GO:0098797    180
GO:0140513    172
GO:0016747    153
GO:0003006    151
GO:1901700    142
GO:0016836    140
GO:0008757    135
GO:0016811    130
GO:0051241    130
GO:0016773    124
GO:0051240    124
GO:0008168    123
GO:0014070    123
GO:1901701    123
Name: count, dtype: int64

In [37]:
df_top20 = df.copy()
is_a_of_interest_top20 = ['GO:0110165', 'GO:0016616', 'GO:0032991', 'GO:0016709', 'GO:0016758', 'GO:0048856', 'GO:0098797', 'GO:0140513', 'GO:0016747', 'GO:0003006', 'GO:1901700', 'GO:0016836', 'GO:0008757',
 'GO:0016811', 'GO:0051241', 'GO:0016773', 'GO:0051240', 'GO:0008168', 'GO:0014070', 'GO:1901701']
df_top20['is_a'] = df_top20['is_a'].apply(lambda labels: [label for label in labels if label in is_a_of_interest_top20])
filtered_df_top20 = df_top20[df_top20['is_a'].apply(lambda x: any(item in is_a_of_interest_top20 for item in (x if isinstance(x, list) else [x])))]

#### Converting the labels to vectors for 20 labels

In [57]:
mlb_20 = MultiLabelBinarizer()
y_top20 = mlb_20.fit_transform(filtered_df_top20['is_a'])
y_top20_df = pd.DataFrame(y_top20, columns=mlb_20.classes_) 

In [58]:
mlb_20.classes_

array(['GO:0003006', 'GO:0008168', 'GO:0008757', 'GO:0014070',
       'GO:0016616', 'GO:0016709', 'GO:0016747', 'GO:0016758',
       'GO:0016773', 'GO:0016811', 'GO:0016836', 'GO:0032991',
       'GO:0048856', 'GO:0051240', 'GO:0051241', 'GO:0098797',
       'GO:0110165', 'GO:0140513', 'GO:1901700', 'GO:1901701'],
      dtype=object)

In [40]:
y_top20_df

,GO:0003006,GO:0008168,GO:0008757,GO:0014070,GO:0016616,GO:0016709,GO:0016747,GO:0016758,GO:0016773,GO:0016811,GO:0016836,GO:0032991,GO:0048856,GO:0051240,GO:0051241,GO:0098797,GO:0110165,GO:0140513,GO:1901700,GO:1901701
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3563,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3564,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3565,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3566,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [41]:
y_top20_df.columns

Index(['GO:0003006', 'GO:0008168', 'GO:0008757', 'GO:0014070', 'GO:0016616',
       'GO:0016709', 'GO:0016747', 'GO:0016758', 'GO:0016773', 'GO:0016811',
       'GO:0016836', 'GO:0032991', 'GO:0048856', 'GO:0051240', 'GO:0051241',
       'GO:0098797', 'GO:0110165', 'GO:0140513', 'GO:1901700', 'GO:1901701'],
      dtype='object')

#### Converting definition attribute to feature vectors for 20 labels

Excluding the words which appear in less than 1% of definitions

In [59]:
vectorizer_20 = CountVectorizer(stop_words='english', min_df = 0.01)
X_tfidf_20 = vectorizer_20.fit_transform(filtered_df_top20['definition'])
X_df_top20 = pd.DataFrame(X_tfidf_20.toarray(), columns=vectorizer_20.get_feature_names_out())


In [60]:
X_df_top20

,acceptor,acetate,acetyl,acid,actin,activates,activity,acyl,adenosyl,adp,...,time,tissue,trans,transcription,transfer,transport,type,udp,wall,yeast
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3563,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3564,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3565,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3566,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


### Random Forests

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X_df_top20, y_top20_df, test_size=0.2, random_state=42)

In [62]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("F1-Score:", str(f1_score(y_test, y_pred_rf, average = 'micro')))
print(classification_report(y_test, y_pred_rf, zero_division=0))

F1-Score: 0.795146324054247
              precision    recall  f1-score   support

           0       0.78      0.55      0.64        33
           1       0.37      0.54      0.44        24
           2       0.58      0.23      0.33        30
           3       0.56      0.95      0.70        20
           4       1.00      0.97      0.98        66
           5       0.96      1.00      0.98        45
           6       0.97      0.89      0.93        36
           7       0.95      0.82      0.88        49
           8       1.00      0.76      0.87        17
           9       0.79      0.46      0.58        24
          10       0.67      0.86      0.75        28
          11       0.71      0.56      0.62        61
          12       0.90      0.86      0.88        43
          13       0.90      1.00      0.95        27
          14       1.00      0.92      0.96        25
          15       0.88      0.54      0.67        26
          16       0.85      0.86      0.86        81

Random Forest Optimization

In [66]:
rf_model = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': list(range(20,300,50)), 
    'max_depth': list(range(3,60,3)), 
    'min_samples_split': [2, 5, 10, 20, 30], 
    'min_samples_leaf': [1, 2, 4, 10]
}
scoring = {'F1-Score': make_scorer(f1_score, average='micro')}
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring=scoring, refit='F1-Score', n_jobs=-1)

grid_search.fit(X_train, y_train)
print('Best parameters found: \n', grid_search.best_params_)

/Users/kajolpatel/Desktop/Individual_Project/MultiGOClassifier/.venv/lib/python3.11/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best parameters found: 
 {'max_depth': 33, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 270}


Testing tuned parameters

In [54]:
rf_model = RandomForestClassifier(max_depth=33, n_estimators=270, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("F1-Score:", str(f1_score(y_test, y_pred_rf, average = 'micro')))
print(classification_report(y_test, y_pred_rf, zero_division=0))

F1-Score: 0.7994330262225372
              precision    recall  f1-score   support

           0       0.78      0.55      0.64        33
           1       0.37      0.54      0.44        24
           2       0.58      0.23      0.33        30
           3       0.54      0.95      0.69        20
           4       1.00      0.97      0.98        66
           5       0.96      1.00      0.98        45
           6       0.97      0.89      0.93        36
           7       0.95      0.86      0.90        49
           8       1.00      0.76      0.87        17
           9       0.79      0.46      0.58        24
          10       0.67      0.86      0.75        28
          11       0.67      0.57      0.62        61
          12       0.93      0.86      0.89        43
          13       0.90      1.00      0.95        27
          14       1.00      0.92      0.96        25
          15       0.93      0.54      0.68        26
          16       0.85      0.93      0.89        8

#### Support Vector Machines

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X_df_top20, y_top20_df, test_size=0.2, random_state=42)

In [48]:
svm_model = OneVsRestClassifier(SVC(random_state=42))
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
print(f1_score(y_test, y_pred_svm, average='micro'))
print(classification_report(y_test, y_pred_svm, zero_division=0))

0.7964860907759883
              precision    recall  f1-score   support

           0       1.00      0.42      0.60        33
           1       1.00      0.08      0.15        24
           2       0.62      0.77      0.69        30
           3       0.50      0.95      0.66        20
           4       1.00      0.97      0.98        66
           5       1.00      0.98      0.99        45
           6       0.94      0.83      0.88        36
           7       0.92      0.69      0.79        49
           8       0.92      0.65      0.76        17
           9       0.83      0.42      0.56        24
          10       0.66      0.75      0.70        28
          11       0.76      0.48      0.59        61
          12       0.95      0.93      0.94        43
          13       0.96      1.00      0.98        27
          14       1.00      0.88      0.94        25
          15       1.00      0.50      0.67        26
          16       0.83      0.93      0.88        81
        

SVM Optimization

In [49]:
svm_model = OneVsRestClassifier(SVC(random_state=42))

param_grid = {
    'estimator__C': [0.1, 1, 10, 0.01, 0.5, 5, 50],
    'estimator__kernel': ['linear', 'rbf', 'poly'],
    'estimator__gamma': ['scale', 'auto',0.001, 0.01, 0.1, 1, 10]
}

scoring = {'F1-Score': make_scorer(f1_score, average='micro')}
grid_search = GridSearchCV(svm_model, param_grid, scoring=scoring, refit='F1-Score', cv=5, n_jobs=-1)

# Perform the search
grid_search.fit(X_train, y_train)

# Best parameter set
print('Best parameters found: \n', grid_search.best_params_)

Best parameters found: 
 {'estimator__C': 5, 'estimator__gamma': 0.1, 'estimator__kernel': 'rbf'}


Testing tuned parameters

In [50]:
svm_model = OneVsRestClassifier(SVC(C= 5, gamma = 0.1, random_state=42))
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
print(f1_score(y_test, y_pred_svm, average='micro'))
print(classification_report(y_test, y_pred_svm, zero_division=0))


0.8118393234672304
              precision    recall  f1-score   support

           0       0.81      0.52      0.63        33
           1       0.55      0.25      0.34        24
           2       0.64      0.77      0.70        30
           3       0.51      0.90      0.65        20
           4       1.00      0.97      0.98        66
           5       0.96      0.98      0.97        45
           6       0.91      0.89      0.90        36
           7       0.95      0.80      0.87        49
           8       0.93      0.76      0.84        17
           9       0.83      0.42      0.56        24
          10       0.66      0.82      0.73        28
          11       0.69      0.57      0.62        61
          12       0.93      0.93      0.93        43
          13       0.90      1.00      0.95        27
          14       1.00      0.92      0.96        25
          15       1.00      0.65      0.79        26
          16       0.85      0.91      0.88        81
        